In [ ]:
!pip install --upgrade numpy --target ./python
!pip install --upgrade numexpr --target ./python
!pip install datasets

In [ ]:
from datasets import load_dataset
SETTING = 'vision'
dataset = load_dataset('MMMU/MMMU_Pro', SETTING, split='test')
image = dataset[0]['image']
display(image)

In [ ]:
import sys
sys.path.append(r"./python")

import os
import json
from model import *

#根据时间情况修改index和language值
index =  "wy_demo_image"
embedding_endpoint_name = "cohere.embed-multilingual-v3"

embedding_type = 'bedrock' if embedding_endpoint_name.find('titan') or embedding_endpoint_name.find('cohere') else 'sagemaker'
embeddings = init_embeddings_bedrock(embedding_endpoint_name)

In [ ]:
import sys
sys.path.append(r"./python")

from tqdm import tqdm
import fitz
from PIL import Image
import numpy as np
import base64
from opensearch_multimodel_dataload import add_multimodel_documents
import re
import os
import time

model_name = "anthropic.claude-3-5-sonnet-20240620-v1:0"
# model_name = "anthropic.claude-3-sonnet-20240229-v1:0"
llm = init_model_bedrock(model_name)
text_max_length = 2000

prompt = """
brief the key search terms of the image. 
Keep the key search terms as simple as possible
No preface, just output the key search terms directly.
"""
llm_max_size = 800
display_max_size = 480

texts = []
metadatas = []

for i in tqdm(range(len(dataset))):
    if i < 21:
        continue
    print('sleep 60 sec')
    time.sleep(60)
    image = dataset[i]['image']
    image_id = dataset[i]['id']
    answer = dataset[i]['answer']
    subject = dataset[i]['subject']
    
    print(image.size)
    if image.size[0] > llm_max_size:
        image_for_llm = image.resize((llm_max_size,int(image.size[1]*llm_max_size / image.size[0])))
        print('resize image size:',image_for_llm.size)
    else:
        image_for_llm = image
    rgb_im = image_for_llm.convert('RGB')
    rgb_im.save('for_llm.jpg')
    
    if image.size[0] > display_max_size:
        image_for_display = image.resize((display_max_size,int(image.size[1]*display_max_size / image.size[0])))
        print('store image size:',image_for_display.size)
    else:
        image_for_display = image
    rgb_im = image_for_display.convert('RGB')
    rgb_im.save('for_display.jpg')
    
    with open("for_llm.jpg", "rb") as image_file:
        image_bytes = image_file.read()
        encoded_image = base64.b64encode(image_bytes).decode("utf-8")
        print('for_llm len:',len(encoded_image))
        model_kwargs = {'image': encoded_image,'max_tokens':2048,'language':'english'}
        llm.model_kwargs = model_kwargs
        response = llm(prompt=prompt)
        response = response.strip()
        texts.append(response)
        print('response:',response)
    
    
    with open("for_display.jpg", "rb") as image_file:
        image_bytes = image_file.read()
        encoded_image = base64.b64encode(image_bytes).decode("utf-8")
        print('for_display len:',len(encoded_image))
        base_64_head = 'data:image/jpg;base64,'
        image_code = base_64_head + encoded_image
        
    metadata = {}
    metadata['sentence'] = response[:text_max_length] if len(response) > text_max_length else response
    metadata['id'] = image_id
    metadata['answer'] = answer
    metadata['subject'] = subject
    metadata['image_code'] = image_code
    metadatas.append(metadata)
    os.remove('for_llm.jpg')
    os.remove('for_display.jpg')
    
    if len(texts) > 0:
        if embedding_type == 'bedrock':
            text_embeddings = embeddings.embed_documents([metadata['sentence'] for metadata in metadatas])
        else:
            text_embeddings = embeddings.embed_documents([metadata['sentence'] for metadata in metadatas],chunk_size=10)

        print('texts len:',len(texts))
        print('metadatas len:',len(metadatas))
        print('embeddings len:',len(text_embeddings))
        print('begin to save in vectore store')
        add_multimodel_documents(
            index,
            texts=texts,
            embeddings=text_embeddings,
            metadatas=metadatas
        )
        print('finish save in vectore store:',index)
        texts = []
        metadatas = []
